# NOTE:
1. The data we are using now has been preprocessed. This means every mesh in the `human-body2.zip` has corresponding label, like `height`, `weight`, `gender`, `arm length` and so on.
2. If the data is preprocessed, u can do either way:
   1. Replace the `Models20K` with the new data.
   2. Delete all the files in `data/human-body2/raw/`. Compress the new data as `human-body2.zip` and move it into `data/human-body2/raw/`.
3. If the data is not preprocessed, u can do the following:
   1. Move the original `human-body2.zip` to `data/human-body2/raw/` and extract the `.zip` file there. 
   2. Run the `label-process.ipynb`. This would give us a new `metric.csv` file.
   3. Pick the `.csv` file you want. Here we choose the `matric_standardization.csv`. We rename it and move the `metric.csv` file to the `data/human-body2/raw/`.
   4. In this case, the `human-body2.zip` is still the original version. But it doesn't affect, as we have deleted the mesh with NaN in the `Models20K`.
4. ALWAYS REMEMBER: The content in the `Models20K` does really matter. So, we need to make sure, they are really preprocessed. After preprocessing, there should be 2161 files in total. (2169 before preprocessing) 

In [1]:
import os.path as osp
import os
import time
import random

import matplotlib.pyplot as plt
import numpy as np

from torch import nn
import torch.nn.functional as F
import torch

from dataset import MeshData, DataLoader


data_dir = osp.join('data', 'human-body2')
template_fp = osp.join('template', 'template2.ply')

batch_size = 128

print("Start Loading datas...")
meshdata = MeshData(data_dir, template_fp)
train_loader = DataLoader(meshdata.train_dataset, batch_size=batch_size)
test_loader = DataLoader(meshdata.test_dataset, batch_size=batch_size)
print("Data Loading finishes!")

Start Loading datas...
Normalizing the training and testing dataset...
Normalization Done!
Data Loading finishes!


# Build the Network

In [2]:
seed = 3407 # Torch.manual_seed(3407) is all you need

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True


## CoMA Based Predictor

### Different Training Losses

**For L2 Loss**

1. If we just use CoMA structure, after 250 epochs, the loss is around 3
2. If we still use CoMA structure, but we stopped at 130 epochs, the loss is 2.5.
3. If we add some linear layers right after the CoMA structure, after 250 epochs, the loss is around 3.5.
   
**For L1 Loss**
Compared with L2 Loss, L1 Loss would not penalize heavily to the outliers. If we still use CoMA structure, but we use L1 Loss, the result has no much difference with the one using L2 Loss. 

### Different Training Mode

Previously, we tried two different training mode: Separate Training and Combined Training. **Separate Training** means for each attribtues, we design a network to predict it. **Combined Training** means we design a network to predict all the attributes at the same time.

We tried both of them in my thesis work. But the result has no much difference. To make things easier in this Jupyter Notebook, we just use the **Combined Training**.

In [3]:
from models import Enblock
from preprocess import mesh_sampling_method

class GraphPredictor(nn.Module):
    def __init__(self, in_channels, out_channels, edge_index, down_transform, K, **kwargs):
        super(GraphPredictor, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.edge_index = edge_index
        self.down_transform = down_transform
        # self.num_vert used in the last and the first layer of encoder and decoder
        self.num_vert = self.down_transform[-1].size(0)

        # encoder
        self.en_layers = nn.ModuleList()
        for idx in range(len(out_channels)):
            if idx == 0:
                self.en_layers.append(
                    Enblock(in_channels, out_channels[idx], K, **kwargs))
            else:
                self.en_layers.append(
                    Enblock(out_channels[idx - 1], out_channels[idx], K,
                            **kwargs))

        # [height, arm_length, crotch_height, 
        #          chest_circumference, hip_circumference, waist_circumference,
        self.en_layers.append(nn.Linear(self.num_vert * out_channels[-1], 6))

        self.reset_parameters()

    def reset_parameters(self):
        for name, param in self.named_parameters():
            if 'bias' in name:
                nn.init.constant_(param, 0)
            else:
                nn.init.xavier_uniform_(param)

    def forward(self, x):
        for i, layer in enumerate(self.en_layers):
            if i != len(self.en_layers) - 1:
                x = layer(x, self.edge_index[i], self.down_transform[i])
            else:
                x = x.view(-1, layer.weight.size(1))
                predicted_val = layer(x)

        return predicted_val

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ds_factor = [4, 4, 4, 4]
edge_index_list, down_transform_list, up_transform_list = mesh_sampling_method(data_fp=data_dir,
                                                                                   template_fp=template_fp,
                                                                                   ds_factors=ds_factor,
                                                                                   device=device)

in_channels = 3
out_channels = [16, 16, 16, 32]
K = 6

/home/wuliwuli/Desktop/predictor-net-measure/preprocess/mesh_sampling.py:59: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525552411/work/torch/csrc/utils/tensor_new.cpp:230.)
  torch.LongTensor([spmat.tocoo().row,


# Define the Training Loop

In [4]:
def train_epoch(model, device, dataloader, optimizer):
    # Set train mode for both the encoder and the decoder
    model.train()
    train_loss = 0.0
    batch_losses = []

    # Iterate the dataloader
    for idx, data in enumerate(dataloader, 0):
        # Move tensor to the proper device(GPU)
        x = data.x.to(device)
        y = data.y.to(device)

        # predict the value
        predicted_value = model(x)
        
        loss = F.mse_loss(predicted_value, y, reduction='sum')

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print batch loss
        batch_losses.append(loss.item())
        train_loss += loss.item()
    
    return train_loss / len(dataloader.dataset), batch_losses


In [5]:
def eval(model, device, dataloader, fp):
    # Set train mode for both the encoder and the decoder
    model.eval()
    test_loss = 0.0

    attributes_loss = torch.zeros(6).to(device)
    
    # height_std = 107.99, arm_std = 45.986, cross_std = 55.731, 
    # chest_std = 124.099, hip_std = 113.026, waist_std = 144.338
    std_tensor = torch.tensor([107.99, 45.986, 55.731, 124.099, 113.026, 144.338]).to(device)

    # Iterate the dataloader
    for idx, data in enumerate(dataloader, 0):
        # Move tensor to the proper device(GPU)
        x = data.x.to(device)
        y = data.y.to(device)

        # predict the value
        predicted_value = model(x)

        # Calculate the loss. Since the reduction is 'none', the loss is of shape (batch_size, 10)
        loss = F.mse_loss(predicted_value, y, reduction='none')
        
        # we sum it up according to each latent dimension
        attributes_loss = attributes_loss + torch.sum(loss.detach(), dim=0)

    # Calculate the average loss over the entire test set
    test_loss = (attributes_loss / len(dataloader.dataset) * std_tensor).cpu().numpy()
    
    with open(fp, "a+") as f:
        f.write("Height Loss: {:.5f}. Arm Loss: {:.5f}. Crotch Loss: {:.5f}.\n".format(test_loss[0], test_loss[1], test_loss[2]))
        f.write("Chest Loss: {:.5f}. Hip Loss: {:.5f}. Waist Loss: {:.5f}\n".format(test_loss[3], test_loss[4], test_loss[5]))

    print("Height Loss: {:.5f}. Arm Loss: {:.5f}. Crotch Loss: {:.5f}.".format(test_loss[0], test_loss[1], test_loss[2]))
    print("Chest Loss: {:.5f}. Hip Loss: {:.5f}. Waist Loss: {:.5f}\n".format(test_loss[3], test_loss[4], test_loss[5]))
    

    loss_dict = {'Height Loss': test_loss[0], 'Arm Loss': test_loss[1], 'Crotch Loss': test_loss[2],
                 'Chest Loss': test_loss[3], 'Hip Loss': test_loss[4], 'Waist Loss': test_loss[5]}

    return loss_dict

In [6]:
def run(model, train_loader, epochs, optimizer, device, fp):
    total_batch_losses = []

    height_losses, arm_losses, crotch_losses, chest_losses, hip_losses, waist_losses = [], [], [], [], [], []

    for epoch in range(1, epochs + 1):

        # Train the model for one epoch
        t = time.time()
        train_loss, loss_dict = train_epoch(model=model, device=device, dataloader=train_loader, optimizer=optimizer)
        t_duration = time.time() - t
        
        with open(fp, 'a') as f:
            f.write('Epoch: {:03d}/{:03d}, Train Loss: {:.5f}, Duration: {:.5f}\n'.format(epoch, epochs, train_loss, t_duration))
        print('Epoch: {:03d}/{:03d}, Train Loss: {:.5f}, Duration: {:.5f}'.format(epoch, epochs, train_loss, t_duration))
        total_batch_losses = total_batch_losses + loss_dict
        # Evaluate on test set
        test_loss_dict = eval(model=model, device=device, dataloader=test_loader, fp=fp)
        
        height_losses.append(test_loss_dict['Height Loss'])
        arm_losses.append(test_loss_dict['Arm Loss'])
        crotch_losses.append(test_loss_dict['Crotch Loss'])
        chest_losses.append(test_loss_dict['Chest Loss'])
        hip_losses.append(test_loss_dict['Hip Loss'])
        waist_losses.append(test_loss_dict['Waist Loss'])
        
    test_loss_dict = {'Height Loss': height_losses, 'Arm Loss': arm_losses, 'Crotch Loss': crotch_losses,
                        'Chest Loss': chest_losses, 'Hip Loss': hip_losses, 'Waist Loss': waist_losses}

    return total_batch_losses, test_loss_dict


In [7]:
# some plot functions

def plot_train_loss(batch_losses, fp):
    plt.plot(batch_losses)
    plt.title('Training Loss Curve')
    plt.xlabel('Batch')
    plt.ylabel('Loss')
    plt.savefig(fp)
    plt.clf()


def plot_test_loss(test_losses, ylabel, title, fp):
    plt.plot(test_losses)
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel(ylabel)
    plt.savefig(fp)
    plt.clf()

def plot_together_loss(height_loss, weight_loss, gender_accuracy, fp):
    # make a figure with two subplots, one for height, weight, the other for gender
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    # for height and weight, add legend as well
    ax1.plot(height_loss, label="height")
    ax1.plot(weight_loss, label="weight")
    ax1.legend()
    ax1.set_title("Height and Weight Loss")
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Loss")

    #for gender accuracy
    ax2.plot(gender_accuracy)
    ax2.set_title("Gender Accuracy")
    ax2.set_xlabel("Epoch")
    ax2.set_ylabel("Accuracy")

    # save the figure
    plt.savefig(fp)

# Training

In [8]:
GNN_together = GraphPredictor(in_channels=in_channels, 
                               out_channels=out_channels, 
                               edge_index=edge_index_list,
                               down_transform=down_transform_list,
                               K=K).cuda()

optimizer = torch.optim.Adam(GNN_together.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 50

print("Start training...")


log_fp = osp.join("predictor_network", "log.txt")

batch_losses, test_loss_dict = run(GNN_together, train_loader, epochs, optimizer, device, fp=log_fp)

PATH = osp.join("predictor_network", "predictor.pth")
torch.save(GNN_together.state_dict(), PATH)

# TODO: plot the results
# plot the results
# plot_train_loss(batch_losses, "predictor_network/together/train_loss.pdf")
# plot_together_loss(height_test_losses, weight_test_losses, gender_test_losses, "predictor_network/together/test_loss.pdf")

Start training...
Epoch: 001/050, Train Loss: 3.12338, Duration: 1.72365
Height Loss: 11.49199. Arm Loss: 8.52810. Crotch Loss: 8.96580.
Chest Loss: 49.62136. Hip Loss: 48.48763. Waist Loss: 57.39664

Epoch: 002/050, Train Loss: 1.44191, Duration: 0.96886
Height Loss: 9.99752. Arm Loss: 7.99568. Crotch Loss: 6.56875.
Chest Loss: 21.50556. Hip Loss: 33.25151. Waist Loss: 25.75312

Epoch: 003/050, Train Loss: 0.90816, Duration: 0.97856
Height Loss: 8.08706. Arm Loss: 5.70454. Crotch Loss: 5.55415.
Chest Loss: 14.27796. Hip Loss: 19.54047. Waist Loss: 19.89572

Epoch: 004/050, Train Loss: 0.71543, Duration: 0.97727
Height Loss: 7.10611. Arm Loss: 4.84554. Crotch Loss: 5.29361.
Chest Loss: 11.16395. Hip Loss: 13.34611. Waist Loss: 16.84242

Epoch: 005/050, Train Loss: 0.61679, Duration: 0.97881
Height Loss: 7.51465. Arm Loss: 4.30802. Crotch Loss: 4.51789.
Chest Loss: 9.76795. Hip Loss: 10.07273. Waist Loss: 15.93835

Epoch: 006/050, Train Loss: 0.56214, Duration: 1.02666
Height Loss: 6.87

# Appendix

## MLP Predictor

The predictor network is fully MLP. After trying different pairs of hyper-parameters, the result is still not that good. After around 20-30 epochs, the training loss doesn't have much difference. But the testing loss keeps changing up and down. 

I think we need to change the model structure. Maybe we can try to use the graph convolutional network. In this Jupyter Notebook, we will only list the training process of CoMA based predictor.

In [9]:
class MLPNetworkBlock(nn.Module):
    def __init__(self, in_channel, out_channel, leaky_relu_slope=0.2):
        super(MLPNetworkBlock, self).__init__()
        self.linear_layer = nn.Linear(in_channel, out_channel)
        self.leaky_relu_slope = leaky_relu_slope

    def forward(self, x):
        out = self.linear_layer(x)
        out = F.leaky_relu(input=out, negative_slope=self.leaky_relu_slope)
        return out


class MLPPredictor(nn.Module):
    def __init__(self, in_channel, out_channels):
        super(MLPPredictor, self).__init__()
        self.in_channel = in_channel
        self.layers = nn.ModuleList()
        for idx in range(len(out_channels)):
            if idx == 0:
                self.layers.append(MLPNetworkBlock(in_channel=in_channel, out_channel=out_channels[0]))
            else:
                self.layers.append(MLPNetworkBlock(in_channel=out_channels[idx-1], out_channel=out_channels[idx]))

        # Linear layer to generate predicted value
        self.linear_layer = nn.Linear(out_channels[-1], 1)


    def forward(self, x):
        x = x.view(x.shape[0], -1)

        for i, layer in enumerate(self.layers):
            x = layer(x)

        # get the predicted value
        z = self.linear_layer(x)
        return z

# define the model

# in_channel = meshdata.num_nodes * 3
# # out_channels = [2048, 512, 128, 64, 16] Loss around 10 
# # out_channels = [1024, 256, 64, 16, 4] Loss around 10
# out_channels = [4096, 1024, 256, 64, 16] 
# predictor_network = MLPPredictor(in_channel, out_channels).cuda()